In [1]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df = pd.read_csv("dados/VilaMariana03-01-23-06-04-2023.csv", delimiter=';')

df = df.replace(',', '.', regex=True)

numeric_columns = ['Temp. Ins. (C)', 'Temp. Max. (C)', 'Temp. Min. (C)',
                   'Umi. Ins. (%)', 'Umi. Max. (%)', 'Umi. Min. (%)',
                   'Pressao Ins. (hPa)', 'Pressao Max. (hPa)',
                   'Pressao Min. (hPa)', 'Vel. Vento (m/s)',
                   'Dir. Vento (m/s)', 'Raj. Vento (m/s)', 'Chuva (mm)']

df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

media_pressao_ins = df['Pressao Ins. (hPa)'].mean()

print("Média da Pressão Ins.: ", media_pressao_ins)

df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y')

df['Hora (UTC)'] = pd.to_timedelta(df['Hora (UTC)'], unit='h')

df['Data'] = df['Data'] + df['Hora (UTC)']

df['Dia'] = df['Data'].dt.day
df['Mes'] = df['Data'].dt.month
df['Ano'] = df['Data'].dt.year
df['Hora'] = df['Data'].dt.hour

df = df.drop(columns=['Data', 'Hora (UTC)'])

df.head()

In [ ]:
colunas_descartadas = ['Temp. Max. (C)', 'Temp. Min. (C)', 'Temp. Ins. (C)', 'Umi. Max. (%)', 'Umi. Min. (%)', 'Pto Orvalho Ins. (C)',
                      'Pto Orvalho Max. (C)', 'Pto Orvalho Min. (C)',
                      'Pressao Max. (hPa)', 'Pressao Min. (hPa)',
                      'Raj. Vento (m/s)', 'Radiacao (KJ/m²)', 'Chuva (mm)']

X = df.drop(colunas_descartadas, axis=1)

X.head()

In [ ]:
# X['Temp. Ins. (C)'] = X['Temp. Ins. (C)'].fillna(X['Temp. Ins. (C)'].median())
X['Umi. Ins. (%)'] = X['Umi. Ins. (%)'].fillna(X['Umi. Ins. (%)'].median())
X['Pressao Ins. (hPa)'] = X['Pressao Ins. (hPa)'].fillna(X['Pressao Ins. (hPa)'].median())
X['Vel. Vento (m/s)'] = X['Vel. Vento (m/s)'].fillna(X['Vel. Vento (m/s)'].median())
X['Dir. Vento (m/s)'] = X['Dir. Vento (m/s)'].fillna(X['Dir. Vento (m/s)'].median())

In [ ]:
Y = df['Temp. Ins. (C)']

Y = Y.fillna(Y.median())

Y.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
modelo = LinearRegression()

In [ ]:
x_treino, x_teste, y_treino, y_teste = train_test_split(X, Y, test_size=0.3)

In [ ]:
y_treino = y_treino.astype('float64')
y_teste = y_teste.astype('float64')

In [ ]:
modelo.fit(x_treino, y_treino)

In [ ]:
score_percent = modelo.score(x_teste, y_teste) * 100
print(f"A precisão do modelo é de {score_percent:.2f}%")

In [ ]:
import pickle

In [ ]:
pickle.dump(modelo, open('modelo_regressao_linear.sav', 'wb'))